In [1]:
!git lfs install
!git clone https://huggingface.co/hexgrad/Kokoro-82M

Git LFS initialized.
Cloning into 'Kokoro-82M'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 131 (delta 57), reused 0 (delta 0), pack-reused 4 (from 1)
Receiving objects: 100% (131/131), 57.13 KiB | 5.19 MiB/s, done.
Resolving deltas: 100% (57/57), done.
Filtering content: 100% (17/17), 820.18 MiB | 45.28 MiB/s, done.


In [2]:
%cd Kokoro-82M
!apt-get -qq -y install espeak-ng > /dev/null 2>&1
!pip install -q phonemizer torch transformers scipy munch

/content/Kokoro-82M
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.0/567.0 kB 29.1 MB/s eta 0:00:00


In [3]:
from models import build_model
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL = build_model('kokoro-v0_19.pth', device)
VOICE_NAME = [
    'af', # Default voice is a 50-50 mix of Bella & Sarah
    'af_bella', 'af_sarah', 'am_adam', 'am_michael',
    'bf_emma', 'bf_isabella', 'bm_george', 'bm_lewis',
    'af_nicole', 'af_sky',
][0]
VOICEPACK = torch.load(f'voices/{VOICE_NAME}.pt', weights_only=True).to(device)
print(f'Loaded voice: {VOICE_NAME}')

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


Loaded voice: af


In [5]:
from kokoro import generate
text = """We'll be using Claude as our LLM and define a number of custom tools.
While most of our tools will connect to a local sqlite database (and require no additional dependencies),
we will also provide a general web search to the agent using Tavily."""
audio, out_ps = generate(MODEL, text, VOICEPACK, lang=VOICE_NAME[0])
# Language is determined by the first letter of the VOICE_NAME:
# 🇺🇸 'a' => American English => en-us
# 🇬🇧 'b' => British English => en-gb

# 4️⃣ Display the 24khz audio and print the output phonemes
from IPython.display import display, Audio
display(Audio(data=audio, rate=24000, autoplay=True))
print(out_ps)

wiːl biː jˈuːzɪŋ klˈɔːd æz ˌaʊɚɹ ˌɛlˌɛlˈɛm ænd dᵻfˈaɪn ɐ nˈʌmbɚɹ ʌv kˈʌstəm tˈuːlz. wˌaɪl mˈoʊst əv ˌaʊɚ tˈuːlz wɪl kənˈɛkt tʊ ɐ lˈoʊkəl ˌɛskjˈuːlˈaɪt dˈeɪɾəbˌeɪs «ænd ɹᵻkwˈaɪɚ nˈoʊ ɐdˈɪʃənəl dᵻpˈɛndənsiz», wiː wɪl ˈɔːlsoʊ pɹəvˈaɪd ɐ dʒˈɛnɚɹəl wˈɛb sˈɜːtʃ tə ðɪ ˈeɪdʒənt jˈuːzɪŋ tˈeɪvili.
